In [30]:
import pandas as pd
import glob
import os
from pprint import pprint

In [31]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\MT\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'general'])
]


# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'primary'])
]


In [32]:
print("General files:")
for f in general_files:
    print(f)


General files:
C:\Huy Phan\College\VoterTurnout\data\MT\20161108__mt__general__precinct.csv


In [33]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:
C:\Huy Phan\College\VoterTurnout\data\MT\20160607__mt__primary__precinct.csv


In [34]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)

        # Drop rows where 'precinct' is NaN
        if 'precinct' in df.columns:
            df = df[df["precinct"].notna()]

        # Drop rows where it is not president
        if 'office' in df.columns:
            df = df[df["office"] == "PRESIDENT"]
        
        df = df.drop_duplicates()

        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)
# Create a new column "precinct" by combining county and precinct
pri_combined_df["precinct"] = pri_combined_df["county"].astype(str) + pri_combined_df["precinct"].astype(str)
pri_combined_df


,county,precinct,office,district,party,candidate,votes,house_district,senate_district
0,Beaverhead,BeaverheadPRECINCT 01,PRESIDENT,NaN,REP,JEB BUSH,10,72,36
1,Beaverhead,BeaverheadPRECINCT 12,PRESIDENT,NaN,REP,JEB BUSH,16,72,36
2,Beaverhead,BeaverheadPRECINCT 14,PRESIDENT,NaN,REP,JEB BUSH,3,72,36
3,Beaverhead,BeaverheadPRECINCT 17,PRESIDENT,NaN,REP,JEB BUSH,3,72,36
4,Beaverhead,BeaverheadPRECINCT 19,PRESIDENT,NaN,REP,JEB BUSH,0,72,36
...,...,...,...,...,...,...,...,...,...
6169,Yellowstone,Yellowstone55.1,PRESIDENT,NaN,DEM,NO PREFERENCE,2,55,28
6170,Yellowstone,Yellowstone40-45,PRESIDENT,NaN,DEM,NO PREFERENCE,6,40,20
6171,Yellowstone,Yellowstone56.2,PRESIDENT,NaN,DEM,NO PREFERENCE,50,56,28
6172,Yellowstone,Yellowstone56.1,PRESIDENT,NaN,DEM,NO PREFERENCE,5,56,28


In [35]:
pri_combined_df["party"].value_counts(dropna=False)

party
REP    4116
DEM    2058
Name: count, dtype: int64

In [36]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "votes"]]
# primary_data.loc[:, "party"] = primary_data["party"].replace({
#     "President Republican Party": "REP",
#     "President Democratic Party": "DEM",
#     "Democratic Party": "DEM",
#     "Republican Party": "REP"
# })
primary_data = primary_data[primary_data["party"].isin(["DEM", "REP"])] # Analyzing only republican and democratic

primary_data

,precinct,party,candidate,votes
0,BeaverheadPRECINCT 01,REP,JEB BUSH,10
1,BeaverheadPRECINCT 12,REP,JEB BUSH,16
2,BeaverheadPRECINCT 14,REP,JEB BUSH,3
3,BeaverheadPRECINCT 17,REP,JEB BUSH,3
4,BeaverheadPRECINCT 19,REP,JEB BUSH,0
...,...,...,...,...
6169,Yellowstone55.1,DEM,NO PREFERENCE,2
6170,Yellowstone40-45,DEM,NO PREFERENCE,6
6171,Yellowstone56.2,DEM,NO PREFERENCE,50
6172,Yellowstone56.1,DEM,NO PREFERENCE,5


In [37]:
primary_data["party"].value_counts(dropna=False)

party
REP    4116
DEM    2058
Name: count, dtype: int64

In [38]:
#Remove Write-in and Registered Voters
primary_data = primary_data[~primary_data["candidate"].isin(["Registered Voters", "Write-In"])] 
primary_data["candidate"].value_counts(dropna=False)

candidate
NO PREFERENCE      1372
JEB BUSH            686
HILLARY CLINTON     686
TED CRUZ            686
JOHN R. KASICH      686
MARCO RUBIO         686
BERNIE SANDERS      686
DONALD J. TRUMP     686
Name: count, dtype: int64

In [39]:
primary_data = primary_data[~primary_data["candidate"].isin(["NO PREFERENCE"])] 
primary_data["candidate"].unique()
candidate_party_map = (
    primary_data.dropna(subset=["candidate", "party"])
                .set_index("candidate")["party"]
                .to_dict()
)
print(candidate_party_map)

{'JEB BUSH': 'REP', 'HILLARY CLINTON': 'DEM', 'TED CRUZ': 'REP', 'JOHN R. KASICH': 'REP', 'MARCO RUBIO': 'REP', 'BERNIE SANDERS': 'DEM', 'DONALD J. TRUMP': 'REP'}


In [40]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

candidate_column,precinct,pri_dem_CLINTON,pri_dem_SANDERS,pri_rep_BUSH,pri_rep_CRUZ,pri_rep_KASICH,pri_rep_RUBIO,pri_rep_TRUMP
0,BeaverheadPRECINCT 01,27,44,10,18,30,8,78
1,BeaverheadPRECINCT 02,22,50,9,38,23,13,252
2,BeaverheadPRECINCT 03,36,33,11,35,35,18,269
3,BeaverheadPRECINCT 04,10,7,5,13,7,5,76
4,BeaverheadPRECINCT 05,1,1,2,7,5,2,56
...,...,...,...,...,...,...,...,...
687,Yellowstone55.4,131,134,6,33,12,9,250
688,Yellowstone55.5,199,164,11,82,47,15,421
689,Yellowstone56.1,22,34,2,15,8,7,106
690,Yellowstone56.2,274,318,21,138,63,30,882


In [41]:
# Process general files
gen_df_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)

        # Select only president
        if 'office' in df.columns:
            df = df[df["office"] == "President" ]

        gen_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(gen_df_list, ignore_index=True)
# Create a new column "precinct" by combining county and precinct
gen_combined_df["precinct"] = gen_combined_df["county"].astype(str) + gen_combined_df["precinct"].astype(str)
gen_combined_df


,county,precinct,district,office,party,candidate,votes
0,Beaverhead,BeaverheadPRECINCT 01,NaN,President,Democrat,Hillary Clinton,175
1,Beaverhead,BeaverheadPRECINCT 12,NaN,President,Democrat,Hillary Clinton,157
2,Beaverhead,BeaverheadPRECINCT 14,NaN,President,Democrat,Hillary Clinton,125
3,Beaverhead,BeaverheadPRECINCT 17,NaN,President,Democrat,Hillary Clinton,18
4,Beaverhead,BeaverheadPRECINCT 19,NaN,President,Democrat,Hillary Clinton,14
...,...,...,...,...,...,...,...
3425,Yellowstone,Yellowstone55.1,NaN,President,Republican,Donald Trump,385
3426,Yellowstone,Yellowstone40-45,NaN,President,Republican,Donald Trump,626
3427,Yellowstone,Yellowstone56.2,NaN,President,Republican,Donald Trump,2144
3428,Yellowstone,Yellowstone56.1,NaN,President,Republican,Donald Trump,235


In [42]:
general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]
general_data["party"].value_counts(dropna=False)


party
Democrat          686
American Delta    686
Libertarian       686
Green             686
Republican        686
Name: count, dtype: int64

In [43]:
general_data["candidate"].value_counts(dropna=False)

candidate
Hillary Clinton             686
Rocky Roque de la Fuenta    686
Gary Johnson                686
Jill Stein                  686
Donald Trump                686
Name: count, dtype: int64

In [44]:
# general_data["candidate"] = general_data["candidate"].str.extract(r"^([^/&]+)").iloc[:, 0].str.strip()
# general_data = general_data[~general_data["candidate"].isin(["Registered Voters", "Write-In"])] 
general_data["candidate"].value_counts(dropna=False)

candidate
Hillary Clinton             686
Rocky Roque de la Fuenta    686
Gary Johnson                686
Jill Stein                  686
Donald Trump                686
Name: count, dtype: int64

In [45]:
# general_data = general_data[~general_data["candidate"].isin(["YES", "NO"])]
# general_data["party"] = general_data.apply(
#     lambda row: candidate_party_map.get(row["candidate"], row["party"]) if pd.isna(row["party"]) else row["party"],
#     axis=1
# )
general_data.loc[:, "party"] = general_data["party"].replace({
    "Democrat": "DEM",
    "Republican": "REP"
})
general_data = general_data[general_data["party"].isin(["DEM", "REP"])] # Analyzing only republican and democratic
general_data

,precinct,party,candidate,votes
0,BeaverheadPRECINCT 01,DEM,Hillary Clinton,175
1,BeaverheadPRECINCT 12,DEM,Hillary Clinton,157
2,BeaverheadPRECINCT 14,DEM,Hillary Clinton,125
3,BeaverheadPRECINCT 17,DEM,Hillary Clinton,18
4,BeaverheadPRECINCT 19,DEM,Hillary Clinton,14
...,...,...,...,...
3425,Yellowstone55.1,REP,Donald Trump,385
3426,Yellowstone40-45,REP,Donald Trump,626
3427,Yellowstone56.2,REP,Donald Trump,2144
3428,Yellowstone56.1,REP,Donald Trump,235


In [49]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

candidate_column,precinct,gen_dem_CLINTON,gen_rep_TRUMP
0,BeaverheadPRECINCT 01,175,203
1,BeaverheadPRECINCT 02,127,551
2,BeaverheadPRECINCT 03,158,507
3,BeaverheadPRECINCT 04,34,131
4,BeaverheadPRECINCT 05,9,99
...,...,...,...
681,Yellowstone55.4,344,731
682,Yellowstone55.5,501,1027
683,Yellowstone56.1,64,235
684,Yellowstone56.2,849,2144


In [47]:
combined = pd.merge(primary_result, general_result, on="precinct", how="inner")
combined["rep_primary_total"] = combined.filter(like="pri_rep_").sum(axis=1)
combined["dem_primary_total"] = combined.filter(like="pri_dem_").sum(axis=1)
combined["general_total"] = combined.filter(like="gen_").sum(axis=1)
combined

candidate_column,precinct,pri_dem_CLINTON,pri_dem_SANDERS,pri_rep_BUSH,pri_rep_CRUZ,pri_rep_KASICH,pri_rep_RUBIO,pri_rep_TRUMP,gen_dem_CLINTON,gen_rep_TRUMP,rep_primary_total,dem_primary_total,general_total
0,BeaverheadPRECINCT 01,27,44,10,18,30,8,78,175,203,144,71,378
1,BeaverheadPRECINCT 02,22,50,9,38,23,13,252,127,551,335,72,678
2,BeaverheadPRECINCT 03,36,33,11,35,35,18,269,158,507,368,69,665
3,BeaverheadPRECINCT 04,10,7,5,13,7,5,76,34,131,106,17,165
4,BeaverheadPRECINCT 05,1,1,2,7,5,2,56,9,99,72,2,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,Yellowstone55.4,131,134,6,33,12,9,250,344,731,310,265,1075
658,Yellowstone55.5,199,164,11,82,47,15,421,501,1027,576,363,1528
659,Yellowstone56.1,22,34,2,15,8,7,106,64,235,138,56,299
660,Yellowstone56.2,274,318,21,138,63,30,882,849,2144,1134,592,2993


In [50]:
combined.to_csv("MT.csv", index=False)
